# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Импортируем нужные библиотеки ⬇

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Откройте и изучите файл

Файл - users_behavior.csv

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Данные в отличном состоянии

## Разбейте данные на выборки

In [4]:
df_train, df_valid = train_test_split(df, test_size=0.25, random_state=12345) # валидационная выборка (25%)

df_train, df_test = train_test_split(df_train, test_size=0.25, random_state=12345) # тестовая выборка (25%)

In [5]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## Исследуйте модели

### Для начала посмотрим на модель "случайного леса"

In [6]:
best_model = None
best_result = 0
for est in range(1,11):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_forest.fit(features_train, target_train)
    result = model_forest.score(features_valid, target_valid)
    
    if result > best_result:
        best_model = model_forest
        best_result = result
print('Accuracy наилучшей модели на валидационной выборке:', best_result.round(3))

Accuracy наилучшей модели на валидационной выборке: 0.779


Accuracy наилучшей модели на валидационной выборке 0.79

### Теперь посмотрим на модель логистической регрессии

In [7]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_log.fit(features_train, target_train)
result = model_log.score(features_train, target_train)
result.round(3)

0.749

Accuracy наилучшей модели на валидационной выборке оказался чуть меньше - 0.702. Но за то вероятность переобучить модель очень мала

## Проверьте модель на тестовой выборке

In [8]:
model = RandomForestClassifier(random_state=12345, n_estimators=est)
model.fit(features_train, target_train)
result_test = model.score(features_test, target_test)
print('Accuracy наилучшей модели на тестовой выборке:', result_test.round(3))

Accuracy наилучшей модели на тестовой выборке: 0.794


Accuracy наилучшей модели на тестовой выборке - 0.794

## (бонус) Проверьте модели на адекватность

Проверим модель "случйного леса" на адекватность

In [9]:
prediction_valid = model_forest.predict(features_valid)
print(accuracy_score(target_valid, prediction_valid).round(3))

0.779


Проверим модель "логистической регрессии" на адекватность

In [10]:
prediction_valid = model_log.predict(features_valid)
print(accuracy_score(target_valid, prediction_valid).round(3))

0.761
